# Student Loan Risk with Deep Learning

In [51]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [52]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [53]:
# Review the data types associated with the columns
loans_df.dtypes

payment_history           float64
location_parameter        float64
stem_degree_score         float64
gpa_ranking               float64
alumni_success            float64
study_major_code          float64
time_to_completion        float64
finance_workshop_score    float64
cohort_ranking            float64
total_loan_score          float64
financial_aid_score       float64
credit_ranking              int64
dtype: object

In [54]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

credit_ranking
1    855
0    744
Name: count, dtype: int64

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [55]:
# Define the target set y using the credit_ranking column
X=loans_df.drop(columns='credit_ranking')
y=loans_df['credit_ranking']

# Display a sample of y
y

0       0
1       0
2       0
3       1
4       0
       ..
1594    0
1595    1
1596    1
1597    0
1598    1
Name: credit_ranking, Length: 1599, dtype: int64

In [56]:
# Define features set X by selecting all columns but credit_ranking
X= loans_df.drop(["credit_ranking"],axis=1).values

# Review the features DataFrame
X

array([[ 7.4  ,  0.7  ,  0.   , ...,  3.51 ,  0.56 ,  9.4  ],
       [ 7.8  ,  0.88 ,  0.   , ...,  3.2  ,  0.68 ,  9.8  ],
       [ 7.8  ,  0.76 ,  0.04 , ...,  3.26 ,  0.65 ,  9.8  ],
       ...,
       [ 6.3  ,  0.51 ,  0.13 , ...,  3.42 ,  0.75 , 11.   ],
       [ 5.9  ,  0.645,  0.12 , ...,  3.57 ,  0.71 , 10.2  ],
       [ 6.   ,  0.31 ,  0.47 , ...,  3.39 ,  0.66 , 11.   ]])

### Step 3: Split the features and target sets into training and testing datasets.


In [57]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [58]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [59]:
# Define the the number of inputs (features) to the model
number_input_features = len(X_train[0])


# Review the number of features
number_input_features

11

In [60]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1 =  8

# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2 = 5

# Define the number of neurons in the output layer
output=(tf.keras.layers.Dense(units=1, activation="sigmoid"))

In [61]:
# Create the Sequential model instance
nn = tf.keras.models.Sequential()

# Add the first hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Add the second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Add the output layer to the model specifying the number of output neurons and activation function
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

C:\Users\austi\AppData\Roaming\Python\Python310\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [62]:
# Display the Sequential model summary
nn.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_14 (Dense)                │ (None, 8)              │            96 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 5)              │            45 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 1)              │             6 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 147 (588.00 B)

 Trainable params: 147 (588.00 B)

 Non-trainable params: 0 (0.00 B)

### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [71]:
# Compile the Sequential model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [72]:
# Fit the model using 50 epochs and the training data
fit_model = nn.fit(X_train_scaled,y_train,epochs=50)

Epoch 1/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5713 - loss: 0.7190
Epoch 2/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6508 - loss: 0.6664 
Epoch 3/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6736 - loss: 0.6390
Epoch 4/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6504 - loss: 0.6451 
Epoch 5/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6960 - loss: 0.6124 
Epoch 6/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6971 - loss: 0.5980 
Epoch 7/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7063 - loss: 0.5949
Epoch 8/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7144 - loss: 0.5663
Epoch 9/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7173 - loss: 0.5621 
Epoch 10/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7459 - loss: 0.5414 
Epoch 11/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7281 - loss: 0.5443 
Epoch 12/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.73

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [73]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

13/13 - 0s - 12ms/step - accuracy: 0.7575 - loss: 0.5018
Loss: 0.5017579793930054, Accuracy: 0.7574999928474426


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [75]:
# Set the model's file path

file_path = Path("student_loans.keras")
# Export your model to a keras file
nn.save(file_path)

---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [76]:
# Set the model's file path

file_path = Path("student_loans.keras")
# Load the model to a new object
nn_imported = tf.keras.models.load_model(file_path)

### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [82]:
# Make predictions with the test data

predictions = nn_imported(X_test_scaled)
# Display a sample of the predictions
predictions

<tf.Tensor: shape=(400, 1), dtype=float32, numpy=
array([[0.55771387],
       [0.11562118],
       [0.8734031 ],
       [0.43092245],
       [0.27342564],
       [0.91856426],
       [0.47995037],
       [0.7716558 ],
       [0.05556622],
       [0.20286126],
       [0.2486174 ],
       [0.14610684],
       [0.4975291 ],
       [0.16015272],
       [0.13798633],
       [0.16198586],
       [0.3012275 ],
       [0.941492  ],
       [0.1108365 ],
       [0.02500971],
       [0.1220344 ],
       [0.14723113],
       [0.7120264 ],
       [0.4099177 ],
       [0.6425221 ],
       [0.827017  ],
       [0.7545706 ],
       [0.8813877 ],
       [0.45522508],
       [0.11415379],
       [0.81039405],
       [0.5347169 ],
       [0.93157125],
       [0.98889667],
       [0.98655826],
       [0.87770265],
       [0.19513945],
       [0.19695508],
       [0.12900181],
       [0.34195802],
       [0.43757114],
       [0.2134507 ],
       [0.8023054 ],
       [0.9421343 ],
       [0.33504504],
     

In [86]:
# Save the predictions to a DataFrame and round the predictions to binary results
predictions_df = pd.DataFrame({"Predictions": predictions.numpy().flatten()})
predictions_df["Predictions"] = predictions_df["Predictions"].round().astype(int)
print(predictions_df.head())

   Predictions
0            1
1            0
2            1
3            0
4            0


### Step 4: Display a classification report with the y test data and predictions

In [88]:
# Print the classification report with the y test data and predictions
y_pred = predictions_df["Predictions"]
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.75      0.77      0.76       199
           1       0.77      0.74      0.75       201

    accuracy                           0.76       400
   macro avg       0.76      0.76      0.76       400
weighted avg       0.76      0.76      0.76       400



---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.

3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.

**1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.**
To build a recommendation system for student loans, we would need to collect data of different types and categories. I think the most important categories would be Student information, Financial informations, Loan informations, school information, as well as their Career prospects or average salaries in their chosen field. Lastly their historical loan information might be useful if they have taken out loans for a car or something else. All of this data is important because it covers key factors for their loan as well as factors to predict if they will be able to repay their loans and provide a potential return on investment. 




**2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.**
Content-based filtering recommends items based on the characteristics of the items and the user's preferences. In this case, the "items" are the loan options, and the "user preferences" are the student's characteristics and needs.
This method is appropriate because it can match loans to studends based on their individual characteristics and circumstances, it doesnt require data from other users-therefore protecting privacy, and it can explain the reccomendations, which is extremely important for financial decisions such as these.


**3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.**
1: Data privacy and security:
Student loan data is highly sensitive, containing personal and financial information. Ensuring the privacy and security of this data is crucial especially because of regulations such as FERPA that govern the privacy of student information.


2: Changing economic conditions:
The student loan landscape can change rapidly due to economic fluctuations, policy changes, or global events (like a pandemic). Certain jobs are also becoming obsolete with the advancement of AI/ML certain tasks can now be automated lowering the job potential of any candidate.


Conclusion: These challenges are particularly concerning for a student loan recommendation system because they directly impact the accuracy and reliability of the recommendations. Poor recommendations could lead students to take on inappropriate levels of debt, affecting their financial futures significantly. 
